In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [4]:
input_dir = '/Users/bamudamaris/ownCloud2/labs/datasets/gfp_measurement/'
output_dir = '/Users/bamudamaris/ownCloud2/labs/datasets/gfp_measurement/'


In [6]:
# remove problematic data points (since they were repeated on a later time)
os.makedirs(output_dir, exist_ok=True)

# Define the list of gene prefixes
cols = ['agaI', 'eptA', 'fimI', 'ghoS', 'yaeL', 'yeeD', 'yfdQ', 'yfgJ']

# Iterate through the files in the input directory
for p in os.listdir(input_dir):
    if p.startswith('clones_batch1'):
        d1_od = pd.read_excel(os.path.join(input_dir, p), sheet_name='OD600', index_col=0)
        d1_od = d1_od.drop(columns=[col for col in d1_od.columns if any(col.startswith(gene) for gene in cols)])
        
        d1_gfp = pd.read_excel(os.path.join(input_dir, p), sheet_name='gfp', index_col=0)
        d1_gfp = d1_gfp.drop(columns=[col for col in d1_gfp.columns if any(col.startswith(gene) for gene in cols)])
        
        output_file = os.path.join(output_dir, 'modified_' + p)
        with pd.ExcelWriter(output_file) as writer:
            d1_od.to_excel(writer, sheet_name='OD600')
            d1_gfp.to_excel(writer, sheet_name='gfp')
    
    elif p.startswith('clones_batch2'):
        d2_od = pd.read_excel(os.path.join(input_dir, p), sheet_name='OD600', index_col=0)
        d2_od = d2_od.drop(columns=[col for col in d2_od.columns if any(col.startswith(gene) for gene in cols)])
        
        d2_gfp = pd.read_excel(os.path.join(input_dir, p), sheet_name='gfp', index_col=0)
        d2_gfp = d2_gfp.drop(columns=[col for col in d2_gfp.columns if any(col.startswith(gene) for gene in cols)])
        
        output_file = os.path.join(output_dir, 'modified_' + p)
        with pd.ExcelWriter(output_file) as writer:
            d2_od.to_excel(writer, sheet_name='OD600')
            d2_gfp.to_excel(writer, sheet_name='gfp')

In [5]:
# List of filenames (prefixes) for each day's data
filenames = ['clones_batch3_R1_20.06.24.xlsx', 'clones_batch3_R2_21.06.24.xlsx', 'clones_batch3_R3_22.06.24.xlsx']

# Initialize empty lists to store averaged data for each day
od_averages = []
gfp_averages = []

# Loop through each filename
for day_index, filename in enumerate(filenames, start =1):
    # Read OD600 and GFP data from Excel file
    df_od = pd.read_excel(os.path.join(input_dir, filename), sheet_name='OD600', index_col=0)
    df_gfp = pd.read_excel(os.path.join(input_dir, filename), sheet_name='gfp', index_col=0)
    
    # Average out od and gfp of replicates
    df_od.columns = [f"{col.split('_')[0]}_{day_index}" for col in df_od.columns]
    df_od_avg = df_od.groupby(df_od.columns, axis=1).mean()

    # Average out fluorescence of replicates
    df_gfp.columns = [f"{col.split('_')[0]}_{day_index}" for col in df_gfp.columns]
    df_gfp_avg = df_gfp.groupby(df_gfp.columns, axis=1).mean()
    
    # Append averaged data to lists
    od_averages.append(df_od_avg)
    gfp_averages.append(df_gfp_avg)

# Concatenate all days' averaged data into single DataFrames
df_od_combined = pd.concat(od_averages, axis=1)
df_gfp_combined = pd.concat(gfp_averages, axis=1)

# Reorder columns to ensure cadB from each day appears next to each other
df_od_combined = df_od_combined.reindex(sorted(df_od_combined.columns), axis=1)
df_gfp_combined = df_gfp_combined.reindex(sorted(df_gfp_combined.columns), axis=1)


# Create Excel writer object
output_file = os.path.join(output_dir, 'averaged_data_clones3.xlsx')  # Adjust output file path as needed
with pd.ExcelWriter(output_file) as writer:
    # Write OD600 data to 'OD600' sheet
    df_od_combined.to_excel(writer, sheet_name='OD600')

    # Write GFP data to 'gfp' sheet
    df_gfp_combined.to_excel(writer, sheet_name='gfp')

print(f"Data saved to {output_file}")


In [6]:
df_gfp_combined

,Blank_1,DH5a_1,agaI-ctrl_1,agaI-test_1,eptA-ctrl_1,eptA-test_1,fimI-ctrl_1,fimI-test_1,ghoS-ctrl_1,ghoS-test_1,...,yaeL-ctrl_3,yaeL-test_3,yeeD-ctrl_3,yeeD-test_3,yfdQ-ctrl_3,yfdQ-test_3,yfgJ-ctrl_3,yfgJ-test_3,yqiD-ctrl_3,yqiD-test_3
Time,,,,,,,,,,,,,,,,,,,,,
30,624.333333,1161.000000,633.000000,633.000000,634.000000,633.000000,641.000000,624.000000,633.000000,621.000000,...,637.333333,632.000000,642.666667,630.666667,641.000000,644.333333,637.666667,633.000000,637.333333,646.333333
60,603.666667,1157.000000,625.000000,627.666667,612.000000,613.333333,621.333333,611.666667,611.666667,617.666667,...,633.333333,627.666667,630.333333,624.666667,641.000000,633.666667,635.000000,631.000000,629.666667,650.333333
90,608.666667,1133.333333,605.000000,613.666667,613.333333,615.666667,614.666667,612.666667,619.666667,623.333333,...,645.666667,623.000000,627.666667,624.666667,625.333333,615.333333,637.000000,631.000000,627.333333,637.333333
120,605.333333,1148.000000,611.666667,619.333333,611.333333,608.333333,632.000000,604.666667,616.666667,604.666667,...,622.666667,629.666667,631.000000,627.333333,646.666667,633.666667,632.333333,630.000000,626.000000,653.333333
150,607.333333,1136.333333,612.666667,620.666667,613.000000,617.333333,612.000000,614.333333,612.333333,611.666667,...,636.666667,625.333333,632.000000,623.666667,639.000000,620.333333,639.000000,624.666667,629.333333,655.333333
180,613.666667,1132.000000,623.666667,625.000000,609.333333,613.333333,621.000000,612.666667,613.333333,607.333333,...,628.666667,630.666667,636.333333,627.000000,631.666667,631.666667,625.666667,642.666667,629.333333,669.000000
210,607.666667,1128.666667,617.000000,618.333333,616.000000,620.333333,629.666667,611.333333,617.666667,615.666667,...,642.000000,631.333333,628.666667,627.666667,635.000000,632.000000,641.000000,635.666667,629.000000,674.000000
240,609.000000,1138.333333,623.000000,631.333333,620.000000,621.333333,625.666667,623.666667,624.333333,629.666667,...,647.000000,640.000000,644.000000,633.666667,643.666667,637.666667,638.000000,631.333333,636.666667,698.333333
270,608.333333,1141.333333,637.666667,625.666667,622.000000,629.000000,637.666667,631.000000,624.333333,620.000000,...,646.000000,646.000000,653.333333,640.666667,645.000000,641.000000,649.000000,646.666667,626.333333,704.333333


Normalise data and save into a single dataframe 

In [11]:
gfp_data = None

files = ['averaged_data_clones1.xlsx', 'averaged_data_clones2.xlsx', 'averaged_data_clones3.xlsx']
columns_to_exclude = ['Blank', 'DH5a', 'pOT2', 'msfGFP', 'ydeQ-ctrl', 'ydeQ-test']

for file in files:
    od_df = pd.read_excel(os.path.join(input_dir, file), sheet_name = 'OD600', index_col = 0)
    gfp_df = pd.read_excel(os.path.join(input_dir, file), sheet_name = 'gfp', index_col = 0)
    
    # Average out od of replicates
    od_df.columns = [col.split('_')[0] for col in od_df.columns]
    od_averaged = od_df.groupby(od_df.columns, axis=1).mean()

    # Average out fluorescence of replicates
    gfp_df.columns = [col.split('_')[0] for col in gfp_df.columns]
    fluorescence_averaged = gfp_df.groupby(gfp_df.columns, axis=1).mean()

    # Normalize OD600 by the blank 
    normalized_od_blank = od_averaged.sub(od_averaged['Blank'], axis=0)

    # Normalize fluorescence by the blank and OD600
    normalized_flo_blank = fluorescence_averaged.sub(fluorescence_averaged['Blank'], axis=0)
    fluorescence_od = normalized_flo_blank / normalized_od_blank
    
    norm_gfp = fluorescence_od.drop(columns = [col for col in columns_to_exclude if col in fluorescence_od.columns])
                                    
    
    if gfp_data is None:
        gfp_data = norm_gfp
    else:
        gfp_data = pd.merge(gfp_data, norm_gfp, left_index = True, right_index = True, how = 'outer')
        
gfp_data.reindex(sorted(gfp_data.columns), axis=1)


,agaI-ctrl,agaI-test,cadB-ctrl,cadB-test,eptA-ctrl,eptA-test,fimI-ctrl,fimI-test,ghoS-ctrl,ghoS-test,...,yfdQ-ctrl,yfdQ-test,yfgJ-ctrl,yfgJ-test,yhaV-ctrl,yhaV-test,yodB-ctrl,yodB-test,yqiD-ctrl,yqiD-test
Time,,,,,,,,,,,,,,,,,,,,,
30,5500.000000,2681.818182,3263.888889,53703.703704,1789.473684,4714.285714,2888.888889,814.814815,5384.615385,-62.500000,...,1360.000000,7071.428571,76.923077,-2619.047619,631.578947,1642.857143,1625.000000,10560.000000,-2500.000000,5800.000000
60,3470.588235,3708.333333,3263.333333,46387.096774,1454.545455,3880.000000,3096.774194,758.620690,6571.428571,1750.000000,...,2038.461538,4800.000000,2062.500000,2619.047619,2515.151515,6312.500000,3080.000000,10500.000000,1263.157895,6913.043478
90,3352.941176,3200.000000,3511.428571,51090.909091,2136.363636,6966.666667,2677.419355,4285.714286,10714.285714,3611.111111,...,3708.333333,3529.411765,7461.538462,3954.545455,1348.837209,6411.764706,785.714286,8857.142857,3050.000000,8956.521739
120,4782.608696,3733.333333,3713.888889,47780.487805,1640.000000,4142.857143,5305.555556,2676.470588,9000.000000,2043.478261,...,4545.454545,9238.095238,4650.000000,3807.692308,3312.500000,2083.333333,1419.354839,10513.513514,2034.482759,9677.419355
150,2333.333333,2000.000000,4413.698630,45346.938776,41.666667,3277.777778,333.333333,264.705882,3666.666667,-1142.857143,...,580.645161,5263.157895,3550.000000,-892.857143,1709.090909,3250.000000,2228.571429,13785.714286,807.692308,7636.363636
180,3192.307692,3090.909091,5226.912929,42566.666667,222.222222,3475.000000,2275.000000,512.820513,3400.000000,391.304348,...,2486.486486,5480.000000,-416.666667,2272.727273,1466.666667,1413.793103,2842.105263,13444.444444,448.275862,10820.512821
210,4208.333333,1111.111111,5763.496144,42681.159420,1740.740741,3780.487805,2948.717949,2175.000000,4880.000000,772.727273,...,2026.315789,4583.333333,2640.000000,1200.000000,892.307692,64.516129,953.488372,17354.166667,151.515152,10282.608696
240,3310.344828,3805.555556,6959.899749,39139.534884,2777.777778,4111.111111,3022.727273,2456.521739,6481.481481,3318.181818,...,2095.238095,5571.428571,2545.454545,1372.093023,2438.356164,8294.117647,4227.272727,23192.307692,1382.352941,12611.111111
270,7593.750000,4595.238095,7529.832936,38905.660377,3516.129032,5180.000000,4521.739130,3730.769231,6060.606061,4230.769231,...,4142.857143,6257.142857,3805.555556,3212.765957,734.939759,5875.000000,3566.037736,23034.482759,2625.000000,12936.507937


In [12]:
gfp_data.to_csv(os.path.join(output_dir, 'normalised_gfp_all_clones.tsv'), sep = '\t')